In [1]:
#modules
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from kerastuner.tuners import RandomSearch
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os, random, shutil, logging, datetime, pickle,io, winsound, joblib
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
print("Linear regression model")

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
try:
    #Disable all GPUS
    tf.config.set_visible_devices([], 'GPU'),
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    #Invalid device or cannot modify virtual devices once initialized.
    pass

Linear regression model
Num GPUs Available:  1


In [2]:
def Q2f1(y_test, y_pred2, y_train):
    '''R2 for test set or Q2f1'''
    mean_y_train = np.mean(y_train)
    num, den = 0, 0
    y_test = list(y_test)
    for i in range(len(y_test)):
        num = num + ((y_test[i] - y_pred2[i])**2)
        den = den + ((y_test[i] - mean_y_train)**2)
    r2test = (1 -(num/den))
    print(r2test)
    return r2test[0]

def Q2f2(y_test, y_pred2):
    '''R2 for test set or Q2f2'''
    mean_y_test = np.mean(y_test)
    num, den = 0, 0
    y_test = list(y_test)
    for i in range(len(y_test)):
        num = num + ((y_test[i] - y_pred2[i])**2)
        den = den + ((y_test[i] - mean_y_test)**2)
    r2test = (1 -(num/den))
    print(r2test)
    return r2test[0]

    
def maecr(y_test, y_pred2, y_train):
    y_test = list(y_test)
    
    min1 = y_train.min()
    max1 = y_train.max()
    rang = max1-min1
    
    n = len(y_test)
    m = len(y_pred2)
    ma, sd = 0,0
    for i, j in zip(range(n),range(m)):
        ma = ma + np.abs(y_test[i] - y_pred2[j])
        
    mae = ma/n
    
    for i, j in zip(range(n),range(m)):
        sd = sd + ((np.abs(y_test[i] - y_pred2[j])-mae)**2)
    
    sd1 = np.sqrt(sd/n)
         
    if (mae <= (0.1*rang)) and ((mae+(3*sd1)) <= (0.2*rang)):
        val = 'GOOD'
        
    elif (mae > (0.15*rang)) or ((mae+(3*sd1)) > (0.25*rang)): 
        val = 'BAD'
        
    else:
        val = 'MODERATE'
        
    return val

def maecr_95(y_test, y_pred2, y_train):
    y_test1 = pd.DataFrame()
    for i in y_test.index:
        y_test1 = y_test1.append({'activity' : y_test[i]}, ignore_index = True)
    
    y_pred2_df = pd.DataFrame()
    for i in range(len(y_pred2)):
        y_pred2_df = y_pred2_df.append({'activity' : y_pred2[i]}, ignore_index = True)
        
    
    
    min1 = y_train.min()
    max1 = y_train.max()
    rang = max1-min1
    
    n = len(y_test)
    ma, sd = 0,0
    m1 =  int(0.05*n)
    df_abs = pd.DataFrame()
    
    for i in range(n):
        df_abs = df_abs.append({'Absolute_error': float(np.abs(y_test1.iloc[i,0] - y_pred2_df.iloc[i,0]))},ignore_index=True)
    #print(df_abs)   
    index = df_abs.nlargest(m1,'Absolute_error').index
    
    y_test1 = y_test1.drop(axis=0, index=index)
    y_pred2_df = y_pred2_df.drop(axis=0, index=index)
    index1 = y_pred2_df.index
   
    n = len(index1)
    for i in index1:
        ma = ma + np.abs(y_test1['activity'].loc[i] - y_pred2_df['activity'].loc[i])
        #print(ma)
        
    mae = ma/n
    
    for i in index1:
        sd = sd + ((np.abs(y_test1['activity'].loc[i] - y_pred2_df['activity'].loc[i])-mae)**2)
    
    sd1 = np.sqrt(sd/n)
         
    if (mae <= (0.1*rang)) and ((mae+(3*sd1)) <= (0.2*rang)):
        val = 'GOOD'
        #print(mae, 0.1*rang, mae+(3*sd1),0.2*rang)
    elif (mae > (0.15*rang)) or ((mae+(3*sd1)) > (0.25*rang)): 
        val = 'BAD'
        #print(mae, 0.15*rang, mae+(3*sd1), 0.25*rang)
    else:
        val = 'MODERATE'
        
    return val

def tropsha(y_train, y_test, y_pred1, y_pred2):
    r2_score(y_train, y_pred1)
    av_yta = np.mean(y_test)
    av_ytp = np.mean(y_pred1)
    k1, k2 = 0, 0
    y_train = list(y_train)
    y_pred1 = list(y_pred1)
    sum1, sum2 = 0,0
    for i in range(len(y_train)):
        sum1 = sum1 + (y_train[i]*y_pred1[i])
        sum2 = sum2 + (y_pred1[i]**2)
    k1 = sum1/sum2
    sum1, sum2 = 0,0
    for i in range(len(y_train)):
        sum1 = sum1 + (y_train[i]*y_pred1[i])
        sum2 = sum2 + (y_train[i]**2)
    k2 = sum1/sum2
    y_pred1_1, y_train1_1 = [], []
    for i in range(len(y_train)):
        y_pred1_1.append(y_pred1[i]*k1)
    for i in range(len(y_train)):
        y_train1_1.append(y_train[i]*k2)
    num, den = 0, 0
    for i in range(len(y_train)):
        num = num + ((y_pred1[i] - y_train1_1[i])**2)
        den = den + ((y_pred1[i] - av_ytp)**2)
    r2 = 1 - (num/den)
    num, den = 0, 0
    for i in range(len(y_train)):
        num = num + ((y_train[i] - y_pred1_1[i])**2)
        den = den + ((y_train[i] - av_yta)**2)
    r2_ = 1 - (num/den)
    
    c1 = r2_score(y_train, y_pred1)
    c2 = Q2f1(y_test, y_pred2, y_train)
    c3 = (c1 - r2)/c1 
    c4 = k1
    c5 = np.abs(c1-r2_)
    
    if c1 >0.5 and c2 > 0.6 and c3 < 0.1 and c4 >= 0.85 and c4 <= 1.15 and c5 < 0.3:
        val = 'PASS'
        
    else:
        val = 'FAIL'
        
    return val
    
def SEE(y_train, y_pred1,dn):
    y_train = list(y_train)
    sum = 0
    for i in range(len(y_train)):
        sum = sum + ((y_train[i] - y_pred1[i])**2)
    
    n = len(y_train)-dn-1
    
    sa = np.sqrt(sum/n)
    print("Model Standard Error of Estmation (Training) : ",round(sa,3))

def get_model_summary(model):
    stream = io.StringIO()
    model.summary(print_fn=lambda x: stream.write(x + '\n'))
    summary_string = stream.getvalue()
    stream.close()
    return summary_string

In [3]:
## dataset
dt = pd.read_csv("final_descp.csv")
X1 =  dt.iloc[:,2:]
#y1 = dt[:]['2IC50'].apply(lambda x: x*(1/1000000000))
y = np.log(dt[:]['2IC50'])


stem = input("Enter the file stem (ss/mm)")

if stem == 'ss':
    from sklearn.preprocessing import StandardScaler
    scale = StandardScaler()
    head = X1.columns
    v = scale.fit_transform(X1)
    X = pd.DataFrame(v,columns = head )
    
elif stem == 'mm':
    from sklearn.preprocessing import MinMaxScaler
    scale = MinMaxScaler()
    head = X1.columns
    v = scale.fit_transform(X1)
    X = pd.DataFrame(v,columns = head )
else:
    X = X1


X_tr, X_val, y_tr, y_val = train_test_split( X, y, test_size=0.2)#, random_state = 23)
X_train, X_test, y_train, y_test = train_test_split( X_tr, y_tr, test_size=0.2)#, random_state = 23)
print(X_train.shape, X_val.shape, X_test.shape)


Enter the file stem (ss/mm) mm


(964, 678) (302, 678) (241, 678)


In [4]:
log_name = str(input('Enter the project name'))+stem
try:
    shutil.rmtree(log_name+str('_log'))
except:
    pass
try:
    shutil.rmtree(log_name)
except:
    pass

def build_model(hp):
    activation1=hp.Choice('Activation_functions', ['relu'])#, elu', 'tanh', 'sigmoid'])
    batch_size = hp.Int('batch_size', 16, 32, step=16)
    loss1 = hp.Choice('loss', ['mean_squared_error', 'huber_loss'])
    #metrics1 =  hp.Choice('metrics', ['mean_squared_error', 'huber_loss'])
    model = keras.Sequential()
    model.add(layers.Dense(units=X.shape[1], input_shape=(X.shape[1],)))      
    for i in range(hp.Int('num_layers', 1, 30, step = 1)):
              model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=16 , default=0), activation=activation1))
            
    model.add(layers.Dense(1))
    model.compile(optimizer = hp.Choice('Optimzer', ['SGD', 'RMSprop', 'Adagrad', 'Adam', 'Adamax']),\
        loss=loss1,\
        metrics='mean_squared_error')
    return model


tuner = RandomSearch(
    build_model,
    objective='val_mean_squared_error',
    max_trials=25,
    executions_per_trial=1,
    directory=log_name,
    project_name=log_name)

Enter the project name ReG_


In [5]:
print(time.ctime())
time1 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = f"{log_name}_log/{time1}"
tensorboard = TensorBoard(log_dir=logdir) # added
callback1 = tf.keras.callbacks.EarlyStopping(monitor='mean_squared_error', patience=20)
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs = 250 ,callbacks=[tensorboard, callback1])    
print(time.ctime())      


Trial 25 Complete [00h 00m 28s]
val_mean_squared_error: 6.845282077789307

Best val_mean_squared_error So Far: 3.6254775524139404
Total elapsed time: 00h 40m 14s
INFO:tensorflow:Oracle triggered exit
Sun Feb 14 18:15:34 2021


In [7]:
dt_tt = pd.DataFrame()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='mean_squared_error', patience=20)
for i in tqdm(range(25)):
    string = f'model_{i}_{stem}'
    best_hp = tuner.get_best_hyperparameters(num_trials=25)[i]
    model = build_model(best_hp)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 250, verbose = 0, callbacks = callback1)
    # Predicting the Test set results
    y_pred1 = model.predict(X_train)
    y_pred2 = model.predict(X_val)
    
    try:
        model_summary_string = get_model_summary(model)
        out = open(string + '.txt','w')
        out.write(model_summary_string)
        out.close


        mse2 = round(mean_squared_error(y_val, y_pred2),3)
        mse1 = round(mean_squared_error(y_train, y_pred1),3)
        mae2 = round(mean_absolute_error(y_val, y_pred2),3)
        mae1 = round(mean_absolute_error(y_train, y_pred1),3)
        cod = round(r2_score(y_train, y_pred1),3)
        q2f1 = round(Q2f1(y_val, y_pred2, y_train),3)
        q2f2 = round(Q2f2(y_val, y_pred2),3)
        maec = maecr(y_val, y_pred2, y_train)
        maec95 = maecr_95(y_val, y_pred2, y_train)
        trop = tropsha(y_train, y_val, y_pred1, y_pred2)

        dt_tt = dt_tt.append({'model' : 'model_'+str(i),\
                              'Activation_functions' : best_hp.get('Activation_functions'),\
                              'Optimzer' : best_hp.get('Optimzer'),\
                                'loss' : best_hp.get('loss'),\
                             'batch_size': best_hp.get('batch_size'),\
                             'num_layers' : best_hp.get('num_layers'),\
                             'MSE Train' : mse1, 'MSE Test' : mse2, \
                             'MAE Train ' : mae1, 'MAE Test' : mae2, \
                             'Coefficient of determination(train)': cod,\
                             'Q2f1' : q2f1, 'Q2f2' : q2f2,\
                             'MAE criteria' : maec, \
                            'MAE criteria (95%)' : maec95, \
                             'TROPSA criteria' : trop},\
                            ignore_index = True)
    
        model.save(string+str('.tf'),overwrite=True)
        
    except:
        print(string, 'failed to generate')
        strr = string+str('.tf')+" failed to generate."
        out = open(string + '.txt','w')
        out.write(strr)
        out.close
        
        try: 
            shutil.rmtree(string+str('.tf'))
        except:
            pass
         
dt_tt.to_csv(log_name+stem+'.csv')

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

[0.31610358]
[0.31152397]
[0.31610358]
INFO:tensorflow:Assets written to: model_0_mm.tf\assets


  4%|███▎                                                                               | 1/25 [00:51<20:37, 51.56s/it]

[-0.14425218]
[-0.15191448]
[-0.14425218]
INFO:tensorflow:Assets written to: model_1_mm.tf\assets


  8%|██████▋                                                                            | 2/25 [01:20<17:06, 44.64s/it]

...

 92%|███████████████████████████████████████████████████████████████████████████▍      | 23/25 [37:29<02:54, 87.11s/it]

[-6.153094]
[-6.200993]
[-6.153094]
INFO:tensorflow:Assets written to: model_23_mm.tf\assets


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [38:06<01:12, 72.20s/it]

[-5.6115227]
[-5.6557956]
[-5.6115227]
INFO:tensorflow:Assets written to: model_24_mm.tf\assets


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [38:49<00:00, 93.20s/it]


In [ ]:
'''from ann_visualizer.visualize import ann_viz
model = tf.keras.models.load_model('model_1.tf')
ann_viz(model, view=True, filename='network.gv', title='MyNeural Network')'''